In [ ]:
# pip install geopandas matplotlib mplleaflet
import geopandas as gpd
import os
import io, requests
import joblib

data_path = 'datas'
os.makedirs(data_path, exist_ok=True)

# Bornes de recharge

In [ ]:
file_name = 'consolidation-etalab-schema-irve-statique-v-2.2.0-20230327'
ext = '.json'
url = 'https://www.data.gouv.fr/fr/datasets/r/7eee8f09-5d1b-4f48-a304-5e99e8da1e26'

try: 
    etalab = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
except:

    etalab = gpd.read_file( url)

    # etalab = gpd.read_file(os.path.join(data_path,file_name + ext))
    
    etalab.to_feather(os.path.join(data_path,file_name + '.feather'))

In [ ]:
etalab.info()

In [ ]:
etalab.isna().any()

In [ ]:
etalab.geometry

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a marker cluster layer for the data
bdr = MarkerCluster(name='BDR')
for idx, row in etalab.iterrows():
    popup = folium.Popup(f'<b>Name:</b> {row["nom_station"]}')
    bdr.add_child(folium.Marker(location=[row.geometry.y, row.geometry.x], popup=popup))

In [ ]:
marker_path = 'datas/markers'
os.makedirs(marker_path, exist_ok=True)

joblib.dump(bdr, os.path.join(marker_path,'bornes_rechargement.mrkr'))

# Carte

In [ ]:
# Create a folium map object
map = folium.Map(location= [48.7, 2.3],
                    width='100%', height='100%', left='0%', top='0%', position='relative', 
                    tiles= 'OpenStreetMap',     #OpenStreetMap', Stamen
                    attr=None,    #  None   Terrain, Toner, and Watercolor
                    min_zoom=0, max_zoom=18, zoom_start=9, 
                    min_lat=- 90, max_lat=90, min_lon=- 180, max_lon=180, max_bounds=False, 
                    crs='EPSG3857', control_scale=False, prefer_canvas=False, 
                    no_touch=False, disable_3d=False, png_enabled=False, zoom_control=True)



# Add the marker cluster layer to the map
bdr.add_to(map)
# Add a layer control to the map
folium.LayerControl().add_to(map)

# Display the map in the notebook
map           #.show_in_browser()


# Save map

In [ ]:
maps_path = 'datas/maps'
os.makedirs(maps_path, exist_ok=True)

map.save(os.path.join(maps_path,'carte_bornes_rechargement.html'))